# SETUP

In [1]:
!pip install faiss-cpu deglib
# !pip install --no-binary=deglib --no-deps --force-reinstall deglib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.9/551.9 kB 21.2 MB/s eta 0:00:00


In [2]:
!curl -L -o sift_subset_100k.zip https://static.visual-computing.com/paper/DEG/sift_subset_100k.zip
!unzip sift_subset_100k.zip
!ls sift_subset_100k

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0M  100 18.0M    0     0  6187k      0  0:00:02  0:00:02 --:--:-- 6186k
Archive:  sift_subset_100k.zip
   creating: sift_subset_100k/
  inflating: sift_subset_100k/sift_query.fvecs  
  inflating: sift_subset_100k/sift_groundtruth.ivecs  
  inflating: sift_subset_100k/sift_base.fvecs  
sift_base.fvecs  sift_groundtruth.ivecs  sift_query.fvecs


# Load dataset

In [3]:
import numpy as np
import time
import faiss

In [4]:
# Read .fvecs and .ivecs files
def read_fvecs(filename):
    with open(filename, 'rb') as f:
        dim = np.fromfile(f, dtype=np.int32, count=1)[0]
    return np.fromfile(filename, dtype=np.float32).reshape(-1, dim + 1)[:, 1:]

def read_ivecs(filename):
    with open(filename, 'rb') as f:
        dim = np.fromfile(f, dtype=np.int32, count=1)[0]
    return np.fromfile(filename, dtype=np.int32).reshape(-1, dim + 1)[:, 1:]

# Load data
sift_base = read_fvecs("./sift_subset_100k/sift_base.fvecs")              # base vectors
sift_query = read_fvecs("./sift_subset_100k/sift_query.fvecs")            # queries
sift_query_gt = read_ivecs("./sift_subset_100k/sift_groundtruth.ivecs")   # ground truth indices

print(f"Base shape: {sift_base.shape}, Query shape: {sift_query.shape}")

Base shape: (100000, 128), Query shape: (10000, 128)


## Nearest Neighbor Search with FAISS

In [5]:
# Restrict to 1 thread
faiss.omp_set_num_threads(1)

# Get dimensionality
d = sift_base.shape[1]

# Create a FAISS index for L2 distance
start = time.time()
index = faiss.IndexFlatL2(d)
end = time.time()
print(f"FAISS index creation took {end - start:.2f} seconds.")

# Add the base vectors to the index
start = time.time()
index.add(sift_base)
end = time.time()
print(f"FAISS adding data took {end - start:.2f} seconds.")

# Search
k = 100  # number of nearest neighbors
start = time.time()
distances, indices = index.search(sift_query, k)
end = time.time()
print(f"FAISS kNNS (1 thread) took {end - start:.2f} seconds.")
print(f"Nearest neighbor indices shape: {indices.shape}")

FAISS index creation took 0.00 seconds.
FAISS adding data took 0.11 seconds.
FAISS kNNS (1 thread) took 17.32 seconds.
Nearest neighbor indices shape: (10000, 100)


# Approximated Nearest Neighbor Search (4 approaches)

In [6]:
def recall_at_k(retrieved_indices, ground_truth, k=100):
    """
    Computes Recall@k for nearest neighbor search.

    Parameters:
        retrieved_indices: np.ndarray of shape (num_queries, k)
        ground_truth: np.ndarray of shape (num_queries, 1) – true nearest neighbor indices
        k: int – number of top predictions to check

    Returns:
        recall: float – fraction of queries where ground truth was in top-k
    """
    correct = 0
    for i in range(ground_truth.shape[0]):
        if ground_truth[i, 0] in retrieved_indices[i, :k]:
            correct += 1
    return correct / ground_truth.shape[0]

## ANNS with FAISS Inverted File (IVF) Index

In [7]:
def run_faiss_ivf_ann(sift_base, sift_query, sift_query_gt, nlist=100, nprobe=10, k=100):
    """Run FAISS IVFFlat ANN search with timing and recall reporting."""

    # Restrict to 1 thread
    faiss.omp_set_num_threads(1)

    # Dimensionality
    d = sift_base.shape[1]

    # Index creation
    t0 = time.time()
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
    t1 = time.time()
    creation_time = t1 - t0

    # Training
    t0 = time.time()
    index.train(sift_base)
    t1 = time.time()
    training_time = t1 - t0

    # Add vectors
    t0 = time.time()
    index.add(sift_base)
    t1 = time.time()
    add_time = t1 - t0

    # Search
    index.nprobe = nprobe
    t0 = time.time()
    distances, indices = index.search(sift_query, k)
    t1 = time.time()
    search_time = t1 - t0
    qps = len(sift_query) / search_time

    # Recall
    recall = recall_at_k(indices, sift_query_gt, k)

    # Pretty print
    print(
        f"[FAISS-IVF] nlist={nlist:4d}, nprobe={nprobe:3d}, k={k:3d} | "
        f"Recall@{k:<3d}: {recall * 100:6.2f}% | "
        f"QPS: {qps:7.2f} | "
        f"Create: {creation_time:5.2f}s | "
        f"Train: {training_time:5.2f}s | "
        f"Add: {add_time:5.2f}s | "
        f"Search: {search_time:6.2f}s"
    )

    return recall, qps

for nprobe in [1, 2, 5]:
    run_faiss_ivf_ann(sift_base, sift_query, sift_query_gt, nlist=100, nprobe=nprobe, k=100)

[FAISS-IVF] nlist= 100, nprobe=  1, k=100 | Recall@100:  56.84% | QPS: 7779.44 | Create:  0.00s | Train:  0.40s | Add:  0.23s | Search:   1.29s
[FAISS-IVF] nlist= 100, nprobe=  2, k=100 | Recall@100:  76.11% | QPS: 6079.76 | Create:  0.00s | Train:  0.31s | Add:  0.16s | Search:   1.64s
[FAISS-IVF] nlist= 100, nprobe=  5, k=100 | Recall@100:  93.43% | QPS: 2817.35 | Create:  0.00s | Train:  0.25s | Add:  0.15s | Search:   3.55s


## ANNS with FAISS Inverted File with Product Quantization (IVFPQ)

In [8]:
def run_faiss_ivfpq_ann(sift_base, sift_query, sift_query_gt, nlist=100, nprobe=10, k=100, m=16, nbits=8):
    """
    Run FAISS IVFPQ ANN search.
    Parameters:
        - sift_base: database vectors (N, d)
        - sift_query: query vectors (Q, d)
        - sift_query_gt: ground truth indices (Q, 1)
        - nlist: number of coarse clusters
        - nprobe: number of clusters to search
        - k: top-k neighbors to retrieve
        - m: number of subquantizers (must divide d)
        - nbits: number of bits per subquantizer
    Returns:
        - recall@k
        - queries per second
    """
    faiss.omp_set_num_threads(1)
    d = sift_base.shape[1]

    # Index creation
    t0 = time.time()
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)
    t1 = time.time()
    creation_time = t1 - t0

    # Training
    t0 = time.time()
    index.train(sift_base)
    t1 = time.time()
    training_time = t1 - t0

    # Add data
    t0 = time.time()
    index.add(sift_base)
    t1 = time.time()
    add_time = t1 - t0

    # Search
    index.nprobe = nprobe
    t0 = time.time()
    distances, indices = index.search(sift_query, k)
    t1 = time.time()
    search_time = t1 - t0
    qps = len(sift_query) / search_time

    # Recall
    recall = recall_at_k(indices, sift_query_gt, k)

    # Pretty print
    print(
        f"[FAISS-IVFPQ] nlist={nlist:4d}, nprobe={nprobe:3d}, k={k:3d}, m={m:2d}, nbits={nbits:2d} | "
        f"Recall@{k:<3d}: {recall * 100:6.2f}% | "
        f"QPS: {qps:7.2f} | "
        f"Create: {creation_time:5.2f}s | "
        f"Train: {training_time:5.2f}s | "
        f"Add: {add_time:5.2f}s | "
        f"Search: {search_time:6.2f}s"
    )

    return recall, qps

for nprobe in [1, 2, 5]:
    run_faiss_ivfpq_ann(sift_base, sift_query, sift_query_gt, nlist=100, nprobe=nprobe, k=100, m=16, nbits=8)

[FAISS-IVFPQ] nlist= 100, nprobe=  1, k=100, m=16, nbits= 8 | Recall@100:  56.84% | QPS: 11101.75 | Create:  0.00s | Train:  5.44s | Add:  1.17s | Search:   0.90s
[FAISS-IVFPQ] nlist= 100, nprobe=  2, k=100, m=16, nbits= 8 | Recall@100:  76.11% | QPS: 7969.58 | Create:  0.00s | Train:  5.26s | Add:  0.81s | Search:   1.25s
[FAISS-IVFPQ] nlist= 100, nprobe=  5, k=100, m=16, nbits= 8 | Recall@100:  93.42% | QPS: 4043.35 | Create:  0.00s | Train:  6.05s | Add:  0.79s | Search:   2.47s


## ANNS with FAISS Hierarchical Navigable Small World (HNSW)

In [9]:
def run_faiss_hnsw_ann(
    sift_base,
    sift_query,
    sift_query_gt,
    k=100,
    hnsw_m=32,
    ef_construction=200,
    ef_search_list=[16, 32, 64, 128]
):
    """
    Run FAISS HNSW ANN search across multiple ef_search values.
    Parameters:
        - sift_base: (N, d) base vectors
        - sift_query: (Q, d) query vectors
        - sift_query_gt: (Q, 1) ground truth
        - k: top-k neighbors to retrieve
        - hnsw_m: number of edges per node
        - ef_construction: controls indexing quality/speed
        - ef_search_list: list of efSearch values to benchmark
    Returns:
        - recalls: list of recall@k values
        - qps_list: list of queries/sec values
    """
    faiss.omp_set_num_threads(1)
    d = sift_base.shape[1]

    # Create HNSW index
    t0 = time.time()
    index = faiss.IndexHNSWFlat(d, hnsw_m, faiss.METRIC_L2)
    index.hnsw.efConstruction = ef_construction
    t1 = time.time()
    creation_time = t1 - t0

    # Add base vectors
    t0 = time.time()
    index.add(sift_base)
    t1 = time.time()
    add_time = t1 - t0

    recalls = []
    qps_list = []

    for ef in ef_search_list:
        index.hnsw.efSearch = ef

        # Search
        t0 = time.time()
        distances, indices = index.search(sift_query, k)
        t1 = time.time()
        search_time = t1 - t0
        qps = len(sift_query) / search_time

        # Recall
        recall = recall_at_k(indices, sift_query_gt, k)
        recalls.append(recall)
        qps_list.append(qps)

        # Pretty print
        print(
            f"[FAISS-HNSW] m={hnsw_m:3d}, efC={ef_construction:4d}, efS={ef:4d}, k={k:3d} | "
            f"Recall@{k:<3d}: {recall * 100:6.2f}% | "
            f"QPS: {qps:7.2f} | "
            f"Create: {creation_time:5.2f}s | "
            f"Add: {add_time:5.2f}s | "
            f"Search: {search_time:6.2f}s"
        )

    return index, recalls, qps_list

index, recalls, qps = run_faiss_hnsw_ann(
    sift_base, sift_query, sift_query_gt,
    k=100,
    hnsw_m=32,
    ef_construction=16,
    ef_search_list=[8, 16, 32, 64, 128, 256]
)

[FAISS-HNSW] m= 32, efC=  16, efS=   8, k=100 | Recall@100:  80.38% | QPS: 11128.39 | Create:  0.00s | Add:  8.81s | Search:   0.90s
[FAISS-HNSW] m= 32, efC=  16, efS=  16, k=100 | Recall@100:  89.56% | QPS: 6398.19 | Create:  0.00s | Add:  8.81s | Search:   1.56s
[FAISS-HNSW] m= 32, efC=  16, efS=  32, k=100 | Recall@100:  95.59% | QPS: 4882.09 | Create:  0.00s | Add:  8.81s | Search:   2.05s
[FAISS-HNSW] m= 32, efC=  16, efS=  64, k=100 | Recall@100:  98.38% | QPS: 3748.09 | Create:  0.00s | Add:  8.81s | Search:   2.67s
[FAISS-HNSW] m= 32, efC=  16, efS= 128, k=100 | Recall@100:  99.47% | QPS: 2024.98 | Create:  0.00s | Add:  8.81s | Search:   4.94s
[FAISS-HNSW] m= 32, efC=  16, efS= 256, k=100 | Recall@100:  99.79% | QPS: 1346.19 | Create:  0.00s | Add:  8.81s | Search:   7.43s


In [10]:
index, recalls, qps = run_faiss_hnsw_ann(
    sift_base, sift_query, sift_query_gt,
    k=100,
    hnsw_m=32,
    ef_construction=160,
    ef_search_list=[8, 16, 32, 64, 128]
)

[FAISS-HNSW] m= 32, efC= 160, efS=   8, k=100 | Recall@100:  86.40% | QPS: 11496.43 | Create:  0.00s | Add: 49.93s | Search:   0.87s
[FAISS-HNSW] m= 32, efC= 160, efS=  16, k=100 | Recall@100:  94.46% | QPS: 8577.57 | Create:  0.00s | Add: 49.93s | Search:   1.17s
[FAISS-HNSW] m= 32, efC= 160, efS=  32, k=100 | Recall@100:  98.33% | QPS: 5852.80 | Create:  0.00s | Add: 49.93s | Search:   1.71s
[FAISS-HNSW] m= 32, efC= 160, efS=  64, k=100 | Recall@100:  99.61% | QPS: 3078.60 | Create:  0.00s | Add: 49.93s | Search:   3.25s
[FAISS-HNSW] m= 32, efC= 160, efS= 128, k=100 | Recall@100:  99.94% | QPS: 2317.30 | Create:  0.00s | Add: 49.93s | Search:   4.32s


## ANNS with the Dynamic Exploration Graph (DEG)

In [11]:
import deglib
import numpy as np
import time

def run_deg_ann(
    base_vectors,
    query_vectors,
    ground_truth,
    k=100,
    edges_per_vertex=30,
    extend_k=60,
    extend_eps=0.1,
    eps_list=[0.001, 0.01, 0.05, 0.1],
):
    """
    Run DEG ANN search across multiple eps values.

    Parameters:
        - base_vectors: (N, d) base vectors
        - query_vectors: (Q, d) query vectors
        - ground_truth: (Q, k) ground truth indices
        - k: top-k neighbors to retrieve
        - edges_per_vertex: number of edges per vertex in the graph
        - extend_k: candidate pool size during graph build
        - extend_eps: effort during graph build
        - eps_list: list of eps values to benchmark
    Returns:
        - graph: the DEG search graph
        - recalls: list of recall@k values
        - qps_list: list of queries/sec values
    """

    # Ensure C-contiguous arrays
    base_vectors = np.ascontiguousarray(base_vectors, dtype=np.float32)
    query_vectors = np.ascontiguousarray(query_vectors, dtype=np.float32)
    ground_truth = np.ascontiguousarray(ground_truth, dtype=np.int32)

    # Build graph
    t0 = time.time()

    from deglib.distances import Metric
    from deglib.graph import SizeBoundedGraph
    from deglib.builder import EvenRegularGraphBuilder, OptimizationTarget
    from deglib.std import Mt19937

    capacity = base_vectors.shape[0]
    graph = SizeBoundedGraph.create_empty(capacity, base_vectors.shape[1], edges_per_vertex, Metric.L2)
    builder = EvenRegularGraphBuilder(
        graph, None, optimization_target=OptimizationTarget.LowLID, extend_k=extend_k, extend_eps=extend_eps, improve_k=0,
        improve_eps=0, max_path_length=0, swap_tries=0,
        additional_swap_tries=0
    )
    labels = np.arange(base_vectors.shape[0], dtype=np.uint32)
    builder.set_thread_count(1)
    builder.set_batch_size(1)
    builder.add_entry(labels, base_vectors)
    builder.build(callback="progress")

    # graph = deglib.builder.build_from_data(
    #     data=base_vectors,
    #     edges_per_vertex=edges_per_vertex,
    #     extend_k=extend_k,
    #     extend_eps=extend_eps,
    #     improve_k=0,
    #     optimization_target=OptimizationTarget.LowLID,
    #     callback="progress"
    # )
    t1 = time.time()
    build_time = t1 - t0

    recalls = []
    qps_list = []

    for eps in eps_list:
        # Vectorized search
        t0 = time.time()
        results, _ = graph.search(query=query_vectors, eps=eps, k=k, threads=1)
        t1 = time.time()
        search_time = t1 - t0
        qps = len(query_vectors) / search_time

        recall = recall_at_k(results, ground_truth, k)
        recalls.append(recall)
        qps_list.append(qps)

        print(
            f"[DEG] d={edges_per_vertex}, k_ext={extend_k}, eps_ext={extend_eps:.3f}, "
            f"eps={eps:.3f}, k={k:3d} | "
            f"Recall@{k:<3d}: {recall * 100:6.2f}% | "
            f"QPS: {qps:7.2f} | "
            f"Build: {build_time:5.2f}s | "
            f"Search: {search_time:6.2f}s"
        )

    return graph, recalls, qps_list


graph, recalls, qps = run_deg_ann(
    sift_base, sift_query, sift_query_gt,
    k=100,
    edges_per_vertex=16,            # Graph degree (d)
    extend_k=32,                    # Build: candidate pool size
    extend_eps=0.1,                 # Build: search radius parameter
    eps_list=[0.0001, 0.001, 0.01], # Try multiple search radius settings
)


100.00% [############################################################] (100000 / 100000)
[DEG] d=16, k_ext=32, eps_ext=0.100, eps=0.000, k=100 | Recall@100:  98.94% | QPS:  430.70 | Build: 17.33s | Search:  23.22s
[DEG] d=16, k_ext=32, eps_ext=0.100, eps=0.001, k=100 | Recall@100:  98.97% | QPS:  423.76 | Build: 17.33s | Search:  23.60s
[DEG] d=16, k_ext=32, eps_ext=0.100, eps=0.010, k=100 | Recall@100:  99.16% | QPS:  417.41 | Build: 17.33s | Search:  23.96s


In [12]:
import deglib_cpp
print(deglib_cpp.avx_usable())
print(deglib_cpp.sse_usable())

True
True
